In [112]:
import os
import pandas as pd 

In [113]:
# Provide the directory path
directory_path = '/mnt/vast/hpc/csg/hcs2152/ZFR_RNA_Editing/SPRINT/Output/A2I_Editing/5dpf'

# Create an empty DataFrame to store data
merged_df = pd.DataFrame()

In [114]:
# Find all files in the directory with the "_editing.txt" suffix
file_names = [file for file in os.listdir(directory_path) if file.endswith("_editing.txt")]

# Process each file
for file in file_names:
    # Extract sample name from the file
    sample_name = file.replace("_editing.txt", "")

    # Read data from the file
    file_path = os.path.join(directory_path, file)
    dtype_dict = {'chr': str, 'start': int, 'stop': int, 'editing type': str, 'read type': str, 'strand': str, 'coverage': str}
    df = pd.read_csv(file_path, sep='\t', dtype=dtype_dict)


    # Rename the 'coverage' column to include the sample name
    df.rename(columns={'coverage': f'{sample_name} coverage'}, inplace=True)

    # Merge the data into the main DataFrame
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on=['chr', 'start', 'stop', 'editing type', 'read type', 'strand'], how='outer', suffixes=('', f'_{sample_name}'))

In [115]:
# Display the intermediate DataFrame
merged_df.head(100)

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage
0,1,59297,59298,AG,hyper,+,1:5,NaN,NaN,NaN,NaN,NaN
1,1,59301,59302,AG,hyper,+,1:6,NaN,NaN,NaN,NaN,NaN
2,1,59314,59315,AG,hyper,+,1:9,NaN,NaN,NaN,NaN,NaN
3,1,59325,59326,AG,hyper,+,1:10,NaN,NaN,1:18,NaN,NaN
4,1,59333,59334,AG,hyper,+,2:10,NaN,NaN,1:19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,208469,208470,TC,hyper,-,3:33,NaN,NaN,NaN,NaN,NaN
96,1,208472,208473,TC,hyper,-,2:33,NaN,NaN,NaN,1:54,NaN
97,1,208504,208505,TC,hyper,-,5:20,NaN,NaN,NaN,4:38,5:26
98,1,208509,208510,TC,hyper,-,3:19,2:9,NaN,NaN,NaN,2:25


In [116]:
# Fill NaN values with '.'
merged_df.fillna(0, inplace=True)

# Display the updated DataFrame
merged_df.head()

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage
0,1,59297,59298,AG,hyper,+,1:5,0,0,0,0,0
1,1,59301,59302,AG,hyper,+,1:6,0,0,0,0,0
2,1,59314,59315,AG,hyper,+,1:9,0,0,0,0,0
3,1,59325,59326,AG,hyper,+,1:10,0,0,1:18,0,0
4,1,59333,59334,AG,hyper,+,2:10,0,0,1:19,0,0


In [117]:
merged_df

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage
0,1,59297,59298,AG,hyper,+,1:5,0,0,0,0,0
1,1,59301,59302,AG,hyper,+,1:6,0,0,0,0,0
2,1,59314,59315,AG,hyper,+,1:9,0,0,0,0,0
3,1,59325,59326,AG,hyper,+,1:10,0,0,1:18,0,0
4,1,59333,59334,AG,hyper,+,2:10,0,0,1:19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4084753,KZ116067.1,72158,72159,TC,hyper,-,0,0,0,0,0,1:9
4084754,KZ116067.1,72163,72164,TC,hyper,-,0,0,0,0,0,1:9
4084755,KZ116067.1,72164,72165,TC,hyper,-,0,0,0,0,0,1:9
4084756,KZ116067.1,72165,72166,TC,hyper,-,0,0,0,0,0,1:9


In [118]:
coverage_columns = ["Ctrl-04 coverage", "Ctrl-05 coverage", "Ctrl-06 coverage",  "NO-04 coverage", "NO-05 coverage", "NO-06 coverage"
]

In [119]:
# Create new columns for each sample to store the summed coverage (A + B)
for col in coverage_columns:
    merged_df[f"{col}_sum"] = merged_df[col].apply(lambda x: sum(map(int, str(x).split(":"))) if ':' in str(x) else int(x) if str(x) != 'nan' else 0)

In [120]:
merged_df

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage,Ctrl-04 coverage_sum,Ctrl-05 coverage_sum,Ctrl-06 coverage_sum,NO-04 coverage_sum,NO-05 coverage_sum,NO-06 coverage_sum
0,1,59297,59298,AG,hyper,+,1:5,0,0,0,0,0,6,0,0,0,0,0
1,1,59301,59302,AG,hyper,+,1:6,0,0,0,0,0,7,0,0,0,0,0
2,1,59314,59315,AG,hyper,+,1:9,0,0,0,0,0,10,0,0,0,0,0
3,1,59325,59326,AG,hyper,+,1:10,0,0,1:18,0,0,11,0,0,19,0,0
4,1,59333,59334,AG,hyper,+,2:10,0,0,1:19,0,0,12,0,0,20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4084753,KZ116067.1,72158,72159,TC,hyper,-,0,0,0,0,0,1:9,0,0,0,0,0,10
4084754,KZ116067.1,72163,72164,TC,hyper,-,0,0,0,0,0,1:9,0,0,0,0,0,10
4084755,KZ116067.1,72164,72165,TC,hyper,-,0,0,0,0,0,1:9,0,0,0,0,0,10
4084756,KZ116067.1,72165,72166,TC,hyper,-,0,0,0,0,0,1:9,0,0,0,0,0,10


In [121]:
# Check if all samples have a minimum coverage of 10 for each row
acceptable_rows = merged_df.apply(lambda row: all(row[f"{col}_sum"] >= 10 for col in coverage_columns), axis=1)

# Filter the rows
filtered_df = merged_df.loc[acceptable_rows].copy()  # Use .loc to avoid chained indexing

In [122]:
filtered_df

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage,Ctrl-04 coverage_sum,Ctrl-05 coverage_sum,Ctrl-06 coverage_sum,NO-04 coverage_sum,NO-05 coverage_sum,NO-06 coverage_sum
53,1,141515,141516,AG,regular_and_hyper,+,43:57,30:45,46:63,43:60,26:44,29:57,100,75,109,103,70,86
60,1,141638,141639,AG,regular_and_hyper,+,31:37,15:15,23:28,18:18,12:13,17:19,68,30,51,36,25,36
61,1,141639,141640,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,12:13,18:19,73,30,55,36,25,37
62,1,141640,141641,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,13:13,18:19,73,30,55,36,26,37
63,1,141648,141649,AG,regular_and_hyper,+,29:37,12:15,17:28,15:19,8:13,16:20,66,27,45,34,21,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950502,KN150699.1,24761,24762,TC,hyper,-,5:5,6:6,7:7,13:13,5:5,7:7,10,12,14,26,10,14
951090,KZ115990.1,7134,7135,AG,hyper,+,2:25,2:19,4:21,1:18,3:28,2:23,27,21,25,19,31,25
951093,KZ115990.1,7175,7176,AG,hyper,+,2:17,2:19,4:18,1:12,3:28,1:19,19,21,22,13,31,20
951094,KZ115990.1,7184,7185,AG,hyper,+,2:14,2:18,1:17,1:12,1:21,1:15,16,20,18,13,22,16


In [123]:
# Calculate the percentage of edited sites for each sample
for col in coverage_columns:
    filtered_df[f"{col}_editing_percentage"] = filtered_df.apply(lambda row: (int(row[col].split(":")[1]) / (int(row[f"{col}_sum"]) )) * 100 if ':' in str(row[f"{col}"]) else 0, axis=1)

In [124]:
filtered_df

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,...,Ctrl-06 coverage_sum,NO-04 coverage_sum,NO-05 coverage_sum,NO-06 coverage_sum,Ctrl-04 coverage_editing_percentage,Ctrl-05 coverage_editing_percentage,Ctrl-06 coverage_editing_percentage,NO-04 coverage_editing_percentage,NO-05 coverage_editing_percentage,NO-06 coverage_editing_percentage
53,1,141515,141516,AG,regular_and_hyper,+,43:57,30:45,46:63,43:60,...,109,103,70,86,57.000000,60.000000,57.798165,58.252427,62.857143,66.279070
60,1,141638,141639,AG,regular_and_hyper,+,31:37,15:15,23:28,18:18,...,51,36,25,36,54.411765,50.000000,54.901961,50.000000,52.000000,52.777778
61,1,141639,141640,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,...,55,36,25,37,50.684932,50.000000,50.909091,50.000000,52.000000,51.351351
62,1,141640,141641,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,...,55,36,26,37,50.684932,50.000000,50.909091,50.000000,50.000000,51.351351
63,1,141648,141649,AG,regular_and_hyper,+,29:37,12:15,17:28,15:19,...,45,34,21,36,56.060606,55.555556,62.222222,55.882353,61.904762,55.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950502,KN150699.1,24761,24762,TC,hyper,-,5:5,6:6,7:7,13:13,...,14,26,10,14,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
951090,KZ115990.1,7134,7135,AG,hyper,+,2:25,2:19,4:21,1:18,...,25,19,31,25,92.592593,90.476190,84.000000,94.736842,90.322581,92.000000
951093,KZ115990.1,7175,7176,AG,hyper,+,2:17,2:19,4:18,1:12,...,22,13,31,20,89.473684,90.476190,81.818182,92.307692,90.322581,95.000000
951094,KZ115990.1,7184,7185,AG,hyper,+,2:14,2:18,1:17,1:12,...,18,13,22,16,87.500000,90.000000,94.444444,92.307692,95.454545,93.750000


In [125]:
filtered_df = filtered_df.drop(columns=[f"{col}_sum" for col in coverage_columns], errors='ignore')


In [126]:
filtered_df

,chr,start,stop,editing type,read type,strand,Ctrl-04 coverage,Ctrl-05 coverage,Ctrl-06 coverage,NO-04 coverage,NO-05 coverage,NO-06 coverage,Ctrl-04 coverage_editing_percentage,Ctrl-05 coverage_editing_percentage,Ctrl-06 coverage_editing_percentage,NO-04 coverage_editing_percentage,NO-05 coverage_editing_percentage,NO-06 coverage_editing_percentage
53,1,141515,141516,AG,regular_and_hyper,+,43:57,30:45,46:63,43:60,26:44,29:57,57.000000,60.000000,57.798165,58.252427,62.857143,66.279070
60,1,141638,141639,AG,regular_and_hyper,+,31:37,15:15,23:28,18:18,12:13,17:19,54.411765,50.000000,54.901961,50.000000,52.000000,52.777778
61,1,141639,141640,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,12:13,18:19,50.684932,50.000000,50.909091,50.000000,52.000000,51.351351
62,1,141640,141641,AG,regular_and_hyper,+,36:37,15:15,27:28,18:18,13:13,18:19,50.684932,50.000000,50.909091,50.000000,50.000000,51.351351
63,1,141648,141649,AG,regular_and_hyper,+,29:37,12:15,17:28,15:19,8:13,16:20,56.060606,55.555556,62.222222,55.882353,61.904762,55.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950502,KN150699.1,24761,24762,TC,hyper,-,5:5,6:6,7:7,13:13,5:5,7:7,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
951090,KZ115990.1,7134,7135,AG,hyper,+,2:25,2:19,4:21,1:18,3:28,2:23,92.592593,90.476190,84.000000,94.736842,90.322581,92.000000
951093,KZ115990.1,7175,7176,AG,hyper,+,2:17,2:19,4:18,1:12,3:28,1:19,89.473684,90.476190,81.818182,92.307692,90.322581,95.000000
951094,KZ115990.1,7184,7185,AG,hyper,+,2:14,2:18,1:17,1:12,1:21,1:15,87.500000,90.000000,94.444444,92.307692,95.454545,93.750000


In [127]:
# Save the merged data to a new file
output_filename = os.path.join(directory_path, "SPRINT_ZFR_editing_5dpf.tsv")
filtered_df.to_csv(output_filename, sep='\t', index=False)

print(f"Merged data written to {output_filename}")

Merged data written to /mnt/vast/hpc/csg/hcs2152/ZFR_RNA_Editing/SPRINT/Output/A2I_Editing/5dpf/SPRINT_ZFR_editing_5dpf.tsv
